In [9]:
!pip install pillow-heif
import torch
import requests
import numpy as np
from google.colab import drive
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd
from PIL import Image
import pillow_heif
from pillow_heif import register_heif_opener
import sqlite3
from sqlite3 import Error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
import random
import shutil
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
#class supporting operations from one single img data source folder
class ImgFolderUtils:
  def __init__(self, imgFolderPath):
    self.imgFolderPath = imgFolderPath
    self.imgFiles = [imgFile for imgFile in os.listdir(imgFolderPath) if imgFile.lower().endswith('.jpg')]

  def random_sample_imgs(self, destFolderPath, sample_amount=50, overlapping_folder=None):
      try:
          #need to exclude files from overlapping folders
          overlapping_files = set()
          if overlapping_folder:
              for folder in overlapping_folder:

                  overlapping_files.update(os.listdir(folder))

          non_overlapping_imgs = [img for img in self.imgFiles if img not in overlapping_files]

          #saftey check for sample amt > number of remaining files
          if len(non_overlapping_imgs) < sample_amount:
              sample_amount = len(non_overlapping_imgs)

          sampled_imgs = random.sample(non_overlapping_imgs, sample_amount)
          os.makedirs(destFolderPath, exist_ok=True)

          for img in sampled_imgs:
              shutil.copy(os.path.join(self.imgFolderPath, img), destFolderPath)
          print("finished")
      except Exception as e:
          print(f"error from random_sample_imgs: {e}")

  def create_train_test_split(self, destParentFolder, stratify=False, test_size=0.2, random_state=42, ):
    try:
      train_df, test_df = train_test_split(self.imgFiles, test_size=test_size, random_state=random_state)
      train_folder = os.path.join(destParentFolder, 'train')
      test_folder = os.path.join(destParentFolder, 'test')
      os.makedirs(train_folder, exist_ok=True)
      os.makedirs(test_folder, exist_ok=True)
    except Exception as e:
      print(f"error from create_train_test_split: {e}")
    pass


In [21]:
imgUtilInstance = ImgFolderUtils('/content/drive/MyDrive/Stave_Project/Developers/staveProjectImgsAll')